In [1]:
from collections import *
from typing import List
import functools
import random
import numpy as np

# 步骤
1. 思考如何转换，如果是两个字符匹配，可以逐步匹配 dp[i][j] 表示 s[:i]和p[:j]；或者是否可以用背包，从前往后推
2. 初始化数组，思考是否需要+1个
3. 初始化第一个元素或者行列，往往和其他的不一样
4. 状态转移方程
5. 获得最后一个元素作为答案  

# 常见错误
1. for i in range(1, n+1) 要记得+1, [0] * (W + 1)
2. 一般dp[i][j]都是前i个，不包括i,j，那样的话range都要从1到n+1
3. 初始条件往往不一样，一定要初始化

120. 
Triangle
Given a triangle array, return the minimum path sum from top to bottom.

For each step, you may move to an adjacent number of the row below. More formally, if you are on index i on the current row, you may move to either index i or index i + 1 on the next row.

 

Example 1:

Input: triangle = [[2],[3,4],[6,5,7],[4,1,8,3]]
Output: 11
Explanation: The triangle looks like:
   2
  3 4
 6 5 7
4 1 8 3
The minimum path sum from top to bottom is 2 + 3 + 5 + 1 = 11 (underlined above).
Example 2:

Input: triangle = [[-10]]
Output: -10

In [2]:
class Solution:
    def minimumTotal(self, triangle: List[List[int]]) -> int:
        return self.DP_2(triangle)

    def DP_1(self, triangle):
        """
        从少到多有边界问题，从多到少有剩余问题
        """
        n = len(triangle)
        dp = triangle[-1]
        for i in range(n-2, -1, -1):
            for j in range(i + 1):
                dp[j] = min(dp[j], dp[j+1]) + triangle[i][j]
        return dp[0]

    def DP_2(self, triangle):
        n = len(triangle)
        dp = [0] * n 
        dp[0] = triangle[0][0]
        for i in range(1, n):
            dp[i] = dp[i-1] + triangle[i][i]
            for j in range(i-1, 0, -1):
                dp[j] = min(dp[j-1], dp[j]) + triangle[i][j]
            dp[0] += triangle[i][0]
            # print(dp)
        return min(dp)

    def DP_n(self, triangle):
        n = len(triangle)
        for i in range(n):
            for j in range(i+1):
                if i > 0:
                    if j == 0:
                        triangle[i][j] += triangle[i-1][j]
                    elif j == i:
                        triangle[i][j] += triangle[i-1][j-1]
                    else:
                        triangle[i][j] += min(triangle[i-1][j-1], triangle[i-1][j])
        return min(triangle[-1])

    def dfs(self, triangle):
        n = len(triangle)
        min_path = float("inf")
        def dfs(i, j, s):
            if i == n:
                nonlocal min_path
                min_path = min(min_path, s)
                return 
            for x in [j, j + 1]:
                if 0 <= x < i + 1:
                    dfs(i + 1, x, s + triangle[i][x])
        dfs(0, 0, 0)
        return min_path





In [3]:
triangle = [[2],[3,4],[6,5,7],[4,1,8,3]]
Solution().minimumTotal(triangle)

11

# 背包问题

## 01 背包

最基本的背包问题就是01背包问题（01 knapsack problem）：一共有N件物品，第i（i从1开始）件物品的重量为w[i]，价值为v[i]。在总重量不超过背包承载上限W的情况下，能够装入背包的最大价值是多少？

In [34]:
class Solution:
    def calculateMaxValue01(self, weights, values, W):
        return self.DP_1(weights, values, W)
        # return self.DP_n(weights, values, W)
        
    def DP_n(self, weights, values, W):
        n = len(weights)
        dp = [[0] * (W+1) for _ in range(n)]
        # dp[i][W] means the maximum value of putting 0,1,..,i into the bag that weighs W

        for i in range(n):
            for w in range(W+1):
                # 这里反正最后一行也是0，所以不用管，最好还是管一下
                dp[i][w] = dp[i-1][w] # 先更新为之前的，再更新为新的，这样确保正确
                if w >= weights[i]:
                    dp[i][w] = max(dp[i][w], dp[i-1][w-weights[i]] + values[i])
                
        return dp[-1][W]
    
    def DP_1(self, weights, values, W):
        n = len(weights)
        dp = [0] * (W + 1)
        
        for i in range(n):
            for w in range(W, -1, -1):
                if w >= weights[i]:
                    dp[w] = max(dp[w], dp[w-weights[i]] + values[i]) 
        return dp[W]
    
    
    def DP_1_path(self, weights, values, W):
        n = len(weights)
        dp = [0] * (W + 1)
        paths = [[0] * (W+1) for _ in range(n)]
        items = [[-1] * (W+1) for _ in range(n)]
        for i in range(n):
            for w in range(W, weights[i]-1, -1):
                if dp[w] < dp[w-weights[i]] + values[i]:
                    dp[w] = dp[w-weights[i]] + values[i]
                    paths[i][w] = weights[i]
                    items[i][w] = i 
                    
        result = []
        w = W 
        for i in range(n-1, -1, -1):
            weight = paths[i][w]
            if weight != 0:
                result.append(weight)
            w -= weight
        print(result[::-1])
        
        item = []
        w = W 
        for i in range(n-1, -1, -1):
            p = items[i][w]
            if p != -1:
                item.append(i)
                weight = weights[p]
                w -= weight 
        print(item[::-1])
            
        
                
                

In [36]:
weights = [1,3,5,4,7,9]
values = [2,5,3,6,7,3]
W = 18
# weights = [1,2,3]
# values = [1,2,3]
# W = 5
Solution().DP_1_path(weights, values, W)

[1, 3, 4, 7]
[0, 1, 3, 4]


In [23]:
class Solution:
    def calculateMaxValue01(self, weights, values, W):
        n = len(weights)
        dp = [0] * (W + 1)
        for i in range(n):
            for w in range(W, weights[i] - 1, -1): # remember -1
                dp[w] = max(dp[w], dp[w-weights[i]] + values[i])

        return dp[W]

In [24]:
weights = [1,3,5,4,7,9]
values = [2,5,3,6,7,3]
W = 18

In [25]:
print(Solution().calculateMaxValue01(weights, values, W))
print(Solution().calculateMaxValue01(weights, values, W))

20
20


## 完全背包

In [51]:
class Solution:
    def calculateMaxValueN(self, weights, values, W):
        return self.DP_1(weights, values, W)
        # return self.DP_n(weights, values, W)
        
    def DP_n(self, weights, values, W):
        n = len(weights)
        dp = [[0] * (W + 1) for _ in range(n)]
        for i in range(n):
            for w in range(W+1):
                dp[i][w] = dp[i-1][w]
                if w >= weights[i]:
                    dp[i][w] = max(dp[i][w], dp[i][w-weights[i]] + values[i])
        return dp[-1][W]
    
    def DP_1(self, weights, values, W):
        n = len(weights)
        dp = [0] * (W + 1)
        
        for i in range(n):
            for w in range(W+1):
                if w >= weights[i]:
                    # 这里是w不是i，易错
                    dp[w] = max(dp[w], dp[w-weights[i]] + values[i]) 
        return dp[W]
                    
    def DP_k(self, weights, values, W):
        n = len(weights)
        dp = [0] * (W + 1)
        for i in range(n):
            for w in range(W, -1, -1):
                dp[w] = max(dp[w - k * weights[i]] + k * values[i] for k in range(w // weights[i] + 1))
        return dp[W]

In [58]:
class Solution:
    def calculateMaxValueN(self, weights, values, W):
        n = len(weights)
        dp = [0] * (W + 1)
        for i in range(n):
            for w in range(weights[i], W + 1):
                dp[w] = max(dp[w], dp[w - weights[i]] + values[i])
        return dp[W]
    
    def calculateMaxValueN_path(self, weights, values, W):
        n = len(weights)
        dp = [0] * (W + 1)
        items = [-1] * (W + 1)
        for i in range(n):
            for w in range(weights[i], W + 1):
                new_value = dp[w - weights[i]] + values[i]
                if new_value > dp[w]:
                    dp[w] = new_value 
                    items[w] = i 
        
        item = []
        w = W 
        while w > 0:
            p = items[w]
            if p != -1:
                item.append(p)
                weight = weights[p]
                w -= weight 
        print(item[::-1])
        return dp[W]

In [68]:
# print(Solution().DP_n(weights, values, W))
# print(Solution().DP_1(weights, values, W))
# print(Solution().DP_k(weights, values, W))

weights = [1,3,4,5]
values = [1,4,5,7]
W = 8
print(Solution().calculateMaxValueN_path(weights, values, W))

[1, 3]
11


## 多维度背包

### 如果不能同时发射

In [26]:
def maxHarm(n, M, T, attacks):
    dp = [[0] * (T + 1) for _ in range(M + 1)]
    for i in range(n):
        hi, mi, ti = attacks[i]
        for m in range(mi, M+1):
            for t in range(ti, T+1):
                dp[m][t] = max(dp[m][t], dp[m-mi][t-ti] + hi)
    return dp[-1][-1]

In [27]:
attacks = [
    [10, 8, 5],
    [5, 2, 1],
    [20, 25, 8]
]
n = len(attacks)
M = 100
T = 20
maxHarm(n, M, T, attacks)

100

# 多重背包

In [22]:
def maxHarm(n, M, T, attacks):
    dp = [0] * (M+1)
    for i in range(n):
        hi, mi, ti = attacks[i]
        for m in range(M, -1, -1):
            dp[m] = max(dp[m - k * mi] + k * hi for k in range(min(m // mi + 1, T // ti + 1)))
    return dp[-1]

In [23]:
attacks = [
    [10, 8, 5],
    [5, 2, 1],
    [20, 25, 8]
]
n = len(attacks)
M = 100
T = 20
maxHarm(n, M, T, attacks)

160

In [12]:
# n, m, T = map(int, input().split())
# A, B, C = [], [], []
# for i in range(n):
#     a, b, c = map(int, input().split())
#     A.append(a)
#     B.append(b)
#     C.append(c)
n, m, T = n, M, T 
A = [attacks[j][0] for j in range(len(attacks))]
B = [attacks[j][1] for j in range(len(attacks))]
C = [attacks[j][2] for j in range(len(attacks))]
dp = [0]*(m+1)  # dp初始化为0
for i in range(n):  # 枚举大炮
    for j in range(m, -1, -1):  # 倒序遍历使得一个大炮不能重复更新
        mx = min((m-j)//B[i], T//C[i])  # 计算当前的大炮最多发射几下
        for k in range(1, mx+1):  # 枚举当前的第i门大炮发射几下
            dp[j+k*B[i]] = max(dp[j+k*B[i]], dp[j]+k*A[i])
print(max(dp))

160


416. 
Partition Equal Subset Sum
Given an integer array nums, return true if you can partition the array into two subsets such that the sum of the elements in both subsets is equal or false otherwise.

 

Example 1:

Input: nums = [1,5,11,5]
Output: true
Explanation: The array can be partitioned as [1, 5, 5] and [11].
Example 2:

Input: nums = [1,2,3,5]
Output: false
Explanation: The array cannot be partitioned into equal sum subsets.

In [69]:
class Solution:
    def canPartition(self, nums: List[int]) -> bool:
        s = sum(nums)
        if s % 2 == 1:
            return False 
        
        W = s // 2
        n = len(nums)
        # nums.sort()
        dp = [0] * (W + 1)
        for i in range(n):
            if dp[W] == W:
                return True 
            for w in range(W, -1, -1):
                if w >= nums[i]:
                    dp[w] = max(dp[w], dp[w-nums[i]] + nums[i])

        return False 
    
    def DP1(self, nums):
        s = sum(nums)
        if s % 2 == 1:
            return False 
        
        W = s // 2
        n = len(nums)
        dp = [-float("inf")] * (W + 1) # 一定要是装满的时候，才会加上前面的
        dp[0] = 0
        for i in range(n):
            for w in range(W, nums[i] - 1, -1): # 可以限制到nums[i] - 1
                dp[w] = max(dp[w], dp[w-nums[i]] + nums[i])
        return dp[W] == W
        

    def DPbool(self, nums):
        s = sum(nums)
        if s % 2 == 1:
            return False 
        
        W = s // 2
        n = len(nums)
        dp = [False] * (W + 1) # 一定要是装满的时候，才会加上前面的
        dp[0] = True
        for i in range(n):
            for w in range(W, nums[i] - 1, -1): # 可以限制到nums[i] - 1
                # dp[w] = max(dp[w], dp[w-nums[i]] + nums[i])
                dp[w] = dp[w] or dp[w-nums[i]]
        return dp[W]
        

In [70]:
nums = [1,5,11,5]
Solution().DP1(nums)

True

322. 
Coin Change
You are given an integer array coins representing coins of different denominations and an integer amount representing a total amount of money.

Return the fewest number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return -1.

You may assume that you have an infinite number of each kind of coin.

 

Example 1:

Input: coins = [1,2,5], amount = 11
Output: 3
Explanation: 11 = 5 + 5 + 1
Example 2:

Input: coins = [2], amount = 3
Output: -1
Example 3:

Input: coins = [1], amount = 0
Output: 0

In [39]:
class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        return self.BFS(coins, amount)

    def BFS(self, coins, amount):
        coins = sorted(coins, reverse=True)
        queue = deque([0])
        step = 0
        if amount == 0: return 0
        visited = set() # Use set to store visited
        while queue:
            step += 1
            for _ in range(len(queue)):
                money = queue.popleft()
                for coin in coins:
                    new_money = money + coin 
                    if new_money in visited: # the new variable whether it has been visited
                        continue 
                    visited.add(new_money)
                    if new_money > amount:
                        continue
                    if new_money == amount:
                        return step 
                    
                    queue.append(new_money)
        return -1



    def DP(self, coins, amount):
        W = amount 
        n = len(coins)
        dp = [float("inf")] * (W + 1)
        dp[0] = 0 

        for i in range(n):
            for w in range(coins[i], W + 1):
                dp[w] = min(dp[w], dp[w - coins[i]] + 1)
        
        # print(dp)
        if dp[W] == float("inf"):
            return -1
        return dp[W]
    
    def DP230211(self, coins, amount):
        n = len(coins)
        W = amount 
        dp = [float("inf")] * (W + 1)
        dp[0] = 0
        for i in range(n):
            for w in range(coins[i], W+1):
                dp[w] = min(dp[w], dp[w-coins[i]] + 1)
        if dp[W] == float("inf"):
            return -1
        return dp[W]
    
    
    def MemorySearch(self, coins, amount):
        n = len(coins)
        coins.sort()
        
        @functools.cache
        def search(m):
            if m == 0:
                return 0
            
            res = [float("inf")]
            for c in coins:
                if m < c:
                    break 
                res.append(1 + search(m-c))
            
            return min(res)
        
        
        return search(amount)      
                
            
        

In [51]:
coins = [1,2,5]
amount = 100


In [52]:
%%timeit -n 100
Solution().BFS(coins, amount)

45.3 µs ± 6.56 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
%%timeit -n 100
Solution().DP230211(coins, amount)

53.5 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [54]:
%%timeit -n 100
Solution().MemorySearch(coins, amount)

79.5 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


518. Coin Change II

You are given an integer array coins representing coins of different denominations and an integer amount representing a total amount of money.

Return the number of combinations that make up that amount. If that amount of money cannot be made up by any combination of the coins, return 0.

You may assume that you have an infinite number of each kind of coin.

The answer is guaranteed to fit into a signed 32-bit integer.

 

Example 1:

Input: amount = 5, coins = [1,2,5]
Output: 4
Explanation: there are four ways to make up the amount:
5=5
5=2+2+1
5=2+1+1+1
5=1+1+1+1+1
Example 2:

Input: amount = 3, coins = [2]
Output: 0
Explanation: the amount of 3 cannot be made up just with coins of 2.
Example 3:

Input: amount = 10, coins = [10]
Output: 1

In [76]:
class Solution:
    def change(self, amount: int, coins: List[int]) -> int:
        return self.DP(amount, coins)

    def DP(self, amount, coins):
        n = len(coins)
        W = amount 
        dp = [0] * (W + 1)
        dp[0] = 1
        for i in range(n):
            for w in range(coins[i], W+1):
                dp[w] += dp[w-coins[i]]
            # print(dp)
        
        return dp[W]

In [81]:
coins = [1,2]
amount = 7
Solution().change(amount, coins)

4

# 空间优化 状态压缩

可以在函数里面套函数来做

70. 
Climbing Stairs
You are climbing a staircase. It takes n steps to reach the top.

Each time you can either climb 1 or 2 steps. In how many distinct ways can you climb to the top?

 

Example 1:

Input: n = 2
Output: 2
Explanation: There are two ways to climb to the top.
1. 1 step + 1 step
2. 2 steps
Example 2:

Input: n = 3
Output: 3
Explanation: There are three ways to climb to the top.
1. 1 step + 1 step + 1 step
2. 1 step + 2 steps
3. 2 steps + 1 step
 

Constraints:

1 <= n <= 45

In [45]:
class Solution:
    def climbStairs(self, n: int) -> int:
        # return self.ordered_answer(n)
        return self.memory_search(n)
        # return self.save_memory(n)

    def ordered_answer(self, n):
        dp = [0] * (n + 1)
        dp[0] = 1
        dp[1] = 1
        for w in range(2, n + 1):
            dp[w] = dp[w - 1] + dp[w - 2]
        return dp[n]

    def save_memory(self, n):
        a = 1
        b = 1
        c = a
        for i in range(n - 1):
           a += b
           b = c
           c = a
        return c

    def memory_search(self, n):
        visited = dict()
        visited[0] = 1
        visited[1] = 1
        def search(x):
            if x in visited:
                return visited[x]
            res = search(x-1) + search(x-2)
            visited[x] = res 
            return res 
        return search(n)



    def unordered_answer(self, n):
        W = n
        values = [1, 2]
        dp = [0] * (W + 1)
        dp[0] = 1
        for i in range(len(values)):
            for w in range(values[i], W + 1):
                dp[w] += dp[w-values[i]]
            print(dp)
        return dp[W]
    
    def optimized_search(self, n):
        # dp = [1, 1]
        # for _ in range(n - 1): # 举例，上2层，只要运行一次就可以，所以是n-1 
        #     temp = dp[1]
        #     dp[1] = dp[1] + dp[0]
        #     dp[0] = temp 
        
        # return dp[1]
        
        # dp = [1, 1]
        # for _ in range(n - 1):
        #     dp[0], dp[1] = dp[1], dp[1] + dp[0]
        
        # return dp[1]
    
        l2, l1 = 1, 1
        for _ in range(n - 1):
            l2, l1 = l1, l1 + l2 
        return l1
        
        
            

In [46]:
print(Solution().memory_search(5))
print(Solution().optimized_search(5))

8
8


# 记忆化搜索

139. Word Break

Given a string s and a dictionary of strings wordDict, return true if s can be segmented into a space-separated sequence of one or more dictionary words.

Note that the same word in the dictionary may be reused multiple times in the segmentation.

 

Example 1:

Input: s = "leetcode", wordDict = ["leet","code"]
Output: true
Explanation: Return true because "leetcode" can be segmented as "leet code".
Example 2:

Input: s = "applepenapple", wordDict = ["apple","pen"]
Output: true
Explanation: Return true because "applepenapple" can be segmented as "apple pen apple".
Note that you are allowed to reuse a dictionary word.
Example 3:

Input: s = "catsandog", wordDict = ["cats","dog","sand","and","cat"]
Output: false

In [198]:
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> bool:
        ''' 
        注意index 一定要草稿上写好
        dp[i]是前i个，不包括i，因为i是多出来的一个，要考虑零的时候
        用if 满足条件，True，否则容易不继承之前的犯错
        '''
        W = len(s)
        dp = [False] * (W + 1)
        dp[0] = True 
        
        for w in range(1, W + 1):
            for i in range(1, min(w + 1, 21)):
                sub = s[w - i: w]
                if dp[w - i] and (sub in wordDict):
                    dp[w] = True
            
        return dp[W]
    
    def dp_bag(self, s, wordDict):
        # n = len(s)
        W = len(s)
        dp = [False] * (W + 1)
        dp[0] = True 
        for w in range(1, W+1): # 这里的不同是不是前面成立这里就会成立的，必须额外满足条件 
            for i in range(1, min(W+1, 21)):
                if (dp[w - i] and s[w-i:w] in wordDict):
                    dp[w] = True
                
        return dp[W]
        
    
    def backward(self, s, wordDict):
        n = len(s)
        dp = [False] * (n + 1)
        dp[0] = True 
        
        for i in range(n):
            for j in range(i+1, n+1):
                if s[i:j] in wordDict and dp[i]:
                    dp[j] = True 
        return dp[n]
    
    def search(self, s, wordDict):
        '''
        分成两部分，先是前面的部分判断，从最初开始判断
        后面的部分再判断
        '''
        @functools.lru_cache(None)
        def backtrack(s):
            if not s:
                return True 
            
            n = len(s)
            for i in range(1, n + 1):
                if s[:i] in wordDict and backtrack(s[i:]):
                    return True
            
            return False

        return backtrack(s)
    
    
    def search2(self, s, wordDict):
        results = []
        result = []
        @functools.lru_cache(None)
        def backtrack(s):
            if not s:
                results.append(result[:])
                return True 
            for i in range(1, len(s)+1):
                if s[:i] in wordDict:
                    result.append(s[:i])
                    if backtrack(s[i:]):
                        return True 
                    result.pop()
            return False 
        status = backtrack(s)
        print(results)
        return status
        
        

In [199]:
s = "applepenapple"
wordDict = ["apple","pen"]
Solution().search(s, wordDict)

True

In [200]:
%%timeit -n 100
Solution().dp_bag(s, wordDict)

10.1 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [201]:
# %%timeit -n 100
Solution().search2(s, wordDict)

[['apple', 'pen', 'apple']]


True

241. 
Different Ways to Add Parentheses

Given a string expression of numbers and operators, return all possible results from computing all the different possible ways to group numbers and operators. You may return the answer in any order.

The test cases are generated such that the output values fit in a 32-bit integer and the number of different results does not exceed 104.

 

Example 1:

Input: expression = "2-1-1"
Output: [0,2]
Explanation:
((2-1)-1) = 0 
(2-(1-1)) = 2
Example 2:

Input: expression = "2*3-4*5"
Output: [-34,-14,-10,-10,10]
Explanation:
(2*(3-(4*5))) = -34 
((2*3)-(4*5)) = -14 
((2*(3-4))*5) = -10 
(2*((3-4)*5)) = -10 
(((2*3)-4)*5) = 10
 

Constraints:

1 <= expression.length <= 20
expression consists of digits and the operator '+', '-', and '*'.
All the integer values in the input expression are in the range [0, 99].

In [ ]:
class Solution:
    def diffWaysToCompute(self, expression: str) -> List[int]:
        ''' 
        记忆化搜索或者分治法，需要找到合适的分隔，之前是不用分隔符，现在可以用分隔符
        '''
        s = expression
        if len(s) <= 2:
            return [int(s)]
        
        results = []
        for i in range(len(s)):
            if not s[i].isdigit():
                lefts = self.diffWaysToCompute(s[:i])
                rights = self.diffWaysToCompute(s[i+1:])
                for l in lefts:
                    for r in rights:
                        if s[i] == '+':
                            res = l + r 
                        elif s[i] == '-':
                            res = l - r 
                        else:
                            res = l * r 
                        results.append(res)
        return results


    def S2(self, expression):
        @functools.lru_cache(None)
        def backtrack(s):
            if len(s) <= 2:
                return [int(s)]
            
            # positions = []
            n = len(s)
            results = []
            for i in range(n):
                if not s[i].isdigit():
                    left = backtrack(s[:i])
                    right = backtrack(s[i+1:])
                    for l in left:
                        for r in right:
                            if s[i] == "*":
                                res = l * r
                            elif s[i] == "+":
                                res = l + r 
                            else:
                                res = l - r 
                            results.append(res)
            return results
                        
        results = backtrack(expression)
        # print(results)
        return results


            


294. Flip Game II

You are playing a Flip Game with your friend.

You are given a string currentState that contains only '+' and '-'. You and your friend take turns to flip two consecutive "++" into "--". The game ends when a person can no longer make a move, and therefore the other person will be the winner.

Return true if the starting player can guarantee a win, and false otherwise.

 

Example 1:

Input: currentState = "++++"
Output: true
Explanation: The starting player can guarantee a win by flipping the middle "++" to become "+--+".
Example 2:

Input: currentState = "+"
Output: false
 

Constraints:

1 <= currentState.length <= 60
currentState[i] is either '+' or '-'.

In [111]:
class Solution:
    def canWin(self, currentState: str) -> bool:
        s = currentState
        if '++' not in s:
            # print("Fail")
            return False 

        n = len(s)

        for i in range(n - 1):
            if s[i:i+2] == '++':
                t = s[:i] + '--' + s[i+2:]
                if not self.canWin(t):
                    # if only friend can not win, you can win
                    return True 
        return False 


In [112]:
currentState = "++++"
Solution().canWin(currentState)

True

# 股票

## 固定模板
- dp[i][k][0] 表示第0-i天，还剩k次交易，不持仓时的利润
- dp[i][k][1] 表示第0-i天，还剩k次交易，持仓时的利润
### 1. 初始化
- 数组大小 dp[n][k][2]
- dp[0][k][0] = 0 如果不持仓，初始利润为0
- dp[0][k][1] = - prices[i] 如果持仓，初始利润为负数
### 2. 状态转移方程
- dp[i][k][0] = max(dp[i-1][k][0], dp[i-1][k][1] + p[i])
- dp[i][k][1] = max(dp[i-1][k][1], dp[i-1][k][0] - p[i])

### 3. 结果
- dp[n-1][0][0] 返回不持仓时的利润

### 4. 状态压缩
- 由于i只和i-1有关，可以用 dp[k][0] = max(dp[k][0], dp[k][1] + p[i])的方法
- 这里更新时隐含了他是前一个
- 如果冻结和前两个有关，就再创造一个变量temp


123. 买卖股票的最佳时机 III

给定一个数组，它的第 i 个元素是一支给定的股票在第 i 天的价格。

设计一个算法来计算你所能获取的最大利润。你最多可以完成 两笔 交易。

注意：你不能同时参与多笔交易（你必须在再次购买前出售掉之前的股票）。

 

示例 1:

输入：prices = [3,3,5,0,0,3,1,4]
输出：6
解释：在第 4 天（股票价格 = 0）的时候买入，在第 6 天（股票价格 = 3）的时候卖出，这笔交易所能获得利润 = 3-0 = 3 。
     随后，在第 7 天（股票价格 = 1）的时候买入，在第 8 天 （股票价格 = 4）的时候卖出，这笔交易所能获得利润 = 4-1 = 3 。
示例 2：

输入：prices = [1,2,3,4,5]
输出：4
解释：在第 1 天（股票价格 = 1）的时候买入，在第 5 天 （股票价格 = 5）的时候卖出, 这笔交易所能获得利润 = 5-1 = 4 。   
     注意你不能在第 1 天和第 2 天接连购买股票，之后再将它们卖出。   
     因为这样属于同时参与了多笔交易，你必须在再次购买前出售掉之前的股票。
示例 3：

输入：prices = [7,6,4,3,1] 
输出：0 
解释：在这个情况下, 没有交易完成, 所以最大利润为 0。
示例 4：

输入：prices = [1]
输出：0

## 如何思考
1. 如果只交易一次的话，可以更新买入价格的最小值，和到目前位置的最大利润 （不用知道后面的最大利润）
2. 如果交易两次的话，可以更新第一次交易的利润，再加上第二次交易的利润

In [3]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        buy1 = buy2 = -prices[0]
        sell1 = sell2 = 0
        n = len(prices)
        for i in range(1, n):
            p = prices[i]
            buy1 = max(buy1, -p)
            sell1 = max(sell1, buy1 + p)
            buy2 = max(buy2, sell1 - p)
            sell2 = max(sell2, buy2 + p)

        return sell2

In [4]:
prices = [3,3,5,0,0,3,1,4]
Solution().maxProfit(prices)

6

188. 买卖股票的最佳时机 IV

给定一个整数数组 prices ，它的第 i 个元素 prices[i] 是一支给定的股票在第 i 天的价格。

设计一个算法来计算你所能获取的最大利润。你最多可以完成 k 笔交易。

注意：你不能同时参与多笔交易（你必须在再次购买前出售掉之前的股票）。

 

示例 1：

输入：k = 2, prices = [2,4,1]
输出：2
解释：在第 1 天 (股票价格 = 2) 的时候买入，在第 2 天 (股票价格 = 4) 的时候卖出，这笔交易所能获得利润 = 4-2 = 2 。
示例 2：

输入：k = 2, prices = [3,2,6,5,0,3]
输出：7
解释：在第 2 天 (股票价格 = 2) 的时候买入，在第 3 天 (股票价格 = 6) 的时候卖出, 这笔交易所能获得利润 = 6-2 = 4 。
     随后，在第 5 天 (股票价格 = 0) 的时候买入，在第 6 天 (股票价格 = 3) 的时候卖出, 这笔交易所能获得利润 = 3-0 = 3 。

In [141]:
class Solution:
    def maxProfit(self, k: int, prices: List[int]) -> int:
        n = len(prices)
        # dp = [[[0 for _ in range(2)] for _ in range(k+1)] for _ in range(n)]
        # for j in range(1, k+1):
        #     dp[0][j][1] = -prices[0]

        # for i in range(1, n):
        #     for j in range(1, k + 1):
        #         p = prices[i]
        #         dp[i][j][0] = max(dp[i-1][j][0], dp[i-1][j][1] + p)
        #         dp[i][j][1] = max(dp[i-1][j][1], dp[i-1][j-1][0] - p)

        # return dp[-1][-1][0]

        # 压缩状态
        # 如果只用上一个，就可以直接去掉那个维度，但是要注意中途变量不能被修改了
        dp = [[0 for _ in range(2)] for _ in range(k+1)]
        for j in range(1, k+1):
            dp[j][1] = -prices[0]

        for i in range(1, n):
            for j in range(1, k + 1):
                p = prices[i]
                dp[j][0] = max(dp[j][0], dp[j][1] + p)
                dp[j][1] = max(dp[j][1], dp[j-1][0] - p)

        return dp[-1][0]

In [142]:
k = 2
prices = [3,2,6,5,0,3]
Solution().maxProfit(k, prices)

7

匹配型：逐步匹配可以吗

In [13]:
nums = [1,2,3,2,5,1,4,8]
def equalSubset(nums):
    N = len(nums)
    S = sum(nums)
    if N % 2 == 1 or S % 2 == 1:
        return -1
    
    n = N // 2
    s = S // 2
    nums.sort()
    result = []
    
    def dfs(i, cnt, p):
        if cnt == n:
            if p == s:
                return True 
            return False 
        if p > s:
            return False 
        for k in range(i+1, N):
            result.append(nums[k])
            if dfs(k, cnt + 1, p + nums[k]):
                return True 
            result.pop()
        return False 
    
    dfs(0, 0, 0)
    if result:
        for num in result:
            nums.remove(num)
        return result + sorted(nums)
            
    return -1

In [18]:
arr = [16, 22, 35, 8, 20, 1, 21, 11]
arr = [1, 7, 20, -7, 5, -14, 6, -4]
arr = [0,0,0,0,0,0,1,2,2,3,3,1,1,3]
arr = [0,0,0,0]
# arr = [0]    
equalSubset(arr)

-1

# 圣光
数学方法，反正每次都要用的


In [41]:
class Solution:
    def minLife(self, k, attacks):
        n = len(attacks)
        @functools.cache
        def cost(i, j): # have had i attacks and have used j light
            if j == 0:
                return -sum(attacks[:i])
            if i == 0:
                return -float("inf")
            return max(cost(i-1, j) - attacks[i-1] + j, cost(i-1, j-1))
        
        return max(1, -max(cost(n, j) for j in range(k + 1)))
    
    def minLifeDP(self, k, attacks):
        n = len(attacks)
        dp = [[0] * (k + 1) for _ in range(n+1)]
        for i in range(1, n + 1):
            dp[i][0] = dp[i-1][0] - attacks[i-1]
        for j in range(1, k + 1):
            dp[0][j] = -float("inf")
        for i in range(1, n+1):
            for j in range(1, k+1):
                dp[i][j] = max(dp[i-1][j-1], dp[i-1][j] + j - attacks[i-1])
                
        # for i in dp:
        #     print(i)
        return max(1, -max(dp[n][j] for j in range(k+1)))
    
    def miniLifeMath(self, k, attacks):
        attacks = list(attacks)
        ls = list(enumerate(attacks))
        ls.sort(key=lambda x: x[1]-x[0], reverse=True)
        keys = [x for x, _ in ls[:k]]
        life = 0
        for k in keys:
            attacks[k] = -1
            
        j = 0
        for i in range(len(attacks)):
            if attacks[i] != -1:
                life = life - attacks[i] + j
            else:
                j += 1
        return max(1, -life)

In [42]:
k = 2
attacks = [3,10,7,8]

In [64]:
k = 10
attacks = np.random.randint(1,1000,100)

In [68]:
%%timeit -n 100
Solution().miniLifeMath(k, attacks)

51.1 µs ± 9.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [72]:
%%timeit -n 100
Solution().minLife(k, attacks)

953 µs ± 57.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [71]:
%%timeit -n 100
Solution().minLifeDP(k, attacks)

506 µs ± 29.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 实力差距

In [8]:
def diff(arr, k):
    n = len(arr)
    arr.sort()
    difs = [arr[i+1] - arr[i] for i in range(n-1)]
    difs = [x if x <= k else float("inf") for x in difs]
    m = len(difs)
    dp = [float("inf")] * (m+2)
    dp[0] = difs[0]
    for i in range(1,m+1):
        dp[i] = min(dp[i-1], dp[i-2]+difs[i-1], dp[i-3]+difs[i-2])
    return dp[m]
    
    

In [9]:
diff([81,87,41,59,81,18],30)

23